In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install tensorflow streamlit


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import os


In [11]:
# Paths to your datasets
train_data_dir = "/content/drive/MyDrive/Teeth_Dataset/Training"
val_data_dir = "/content/drive/MyDrive/Teeth_Dataset/Validation"
test_data_dir = "/content/drive/MyDrive/Teeth_Dataset/Testing"

# Define Image Data Generators for Augmentation
train_augment = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2)
])

# Load training dataset with augmentation
train_dataset = image_dataset_from_directory(
    train_data_dir,
    image_size=(128, 128),
    batch_size=32,
    label_mode='categorical',
    shuffle=True
).map(lambda x, y: (train_augment(x), y))

# Load validation and test datasets
validation_dataset = image_dataset_from_directory(
    val_data_dir,
    image_size=(128, 128),
    batch_size=32,
    label_mode='categorical',
    shuffle=False
)

test_dataset = image_dataset_from_directory(
    test_data_dir,
    image_size=(128, 128),
    batch_size=32,
    label_mode='categorical',
    shuffle=False
)


Found 2957 files belonging to 7 classes.
Found 1028 files belonging to 7 classes.
Found 1028 files belonging to 7 classes.


In [12]:
# Print one of the training examples
for images, labels in train_dataset.take(1):
    print("Image shape:", images[0].numpy().shape)
    print("Label:", labels[0].numpy())


Image shape: (128, 128, 3)
Label: [0. 0. 0. 0. 0. 0. 1.]


In [13]:
# Load EfficientNetB0 with pre-trained ImageNet weights
base_model = EfficientNetB0(input_shape=(128, 128, 3), include_top=False, weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Create a new model on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(7, activation='softmax')  # 7 classes for 7 diseases
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
# model.summary()


In [15]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

# Train the model
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=20,
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 67s 468ms/step - accuracy: 0.3701 - loss: 1.6227 - val_accuracy: 0.6780 - val_loss: 0.9064
Epoch 2/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 47s 234ms/step - accuracy: 0.6264 - loss: 1.0190 - val_accuracy: 0.7490 - val_loss: 0.7604
Epoch 3/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 21s 221ms/step - accuracy: 0.6912 - loss: 0.8751 - val_accuracy: 0.7947 - val_loss: 0.6350
Epoch 4/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 45s 259ms/step - accuracy: 0.7230 - loss: 0.7673 - val_accuracy: 0.8035 - val_loss: 0.5832
Epoch 5/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 40s 252ms/step - accuracy: 0.7624 - loss: 0.6763 - val_accuracy: 0.8298 - val_loss: 0.5041
Epoch 6/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 23s 251ms/step - accuracy: 0.7655 - loss: 0.6374 - val_accuracy: 0.8541 - val_loss: 0.4331
Epoch 7/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 41s 248ms/step - accuracy: 0.7925 - loss: 0.5650 - val_accuracy: 0.8609 - val_loss: 0.4081
Epoch 8/20
93/93 ━━━━━━━━━━━━━━━━━━━━ 21s 222ms/step - accuracy: 0.8304 - loss: 0.4923 - val_accu

In [16]:
# Unfreeze the base model for fine-tuning
base_model.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
history_fine = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10,
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 148s 788ms/step - accuracy: 0.3611 - loss: 2.8921 - val_accuracy: 0.7305 - val_loss: 0.8179
Epoch 2/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 25s 267ms/step - accuracy: 0.4128 - loss: 2.4135 - val_accuracy: 0.6051 - val_loss: 1.3244
Epoch 3/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 23s 247ms/step - accuracy: 0.4468 - loss: 2.2573 - val_accuracy: 0.5574 - val_loss: 1.4652
Epoch 4/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 43s 264ms/step - accuracy: 0.4869 - loss: 1.9416 - val_accuracy: 0.5496 - val_loss: 1.4721
Epoch 5/10
93/93 ━━━━━━━━━━━━━━━━━━━━ 23s 247ms/step - accuracy: 0.5078 - loss: 1.8785 - val_accuracy: 0.5720 - val_loss: 1.4188


In [18]:
# Load the best model saved during training
model.load_weights('best_model.keras')

# Evaluate on the test dataset
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.4f}")


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 432 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


33/33 ━━━━━━━━━━━━━━━━━━━━ 339s 10s/step - accuracy: 0.9444 - loss: 0.1584
Test accuracy: 0.9339


In [19]:
# Save the model for later deployment with Streamlit
model.save('teeth_disease_classifier.h5')


In [ ]:
import streamlit as st
import tensorflow as tf
from PIL import Image, ImageOps
import numpy as np

# Load the model
# model = tf.keras.models.load_model('mnist_classifier.h5')

# Define a function to preprocess the image
def preprocess_image(image):
    size = (28, 28)  # MNIST images are 28x28
    image = ImageOps.grayscale(image)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    image = np.asarray(image)
    image = image / 255.0
    image = np.expand_dims(image, axis=-1)  # Add channel dimension
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Streamlit app
st.write("# Disease Classifier")

uploaded_file = st.file_uploader("Choose an image...", type="jpg")

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_column_width=True)
    st.write("")
    st.write("Classifying...")

    # Preprocess the image
    img = preprocess_image(image)

    # Make prediction
    prediction = model.predict(img)
    st.write(f"Prediction: {np.argmax(prediction)}")

In [ ]:
# to run Streamlit but u should do it from vscode 3l4an y4t8l
# streamlit run app.py
#